# FIND USER'S NAME AND ACCOUNT LINKS BASED ON KEYWORD

TODO: add refresh at every error , if the error persists after 3 refresh then quit

#### Import necessary libraries

In [6]:
# for scraping 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from bs4 import BeautifulSoup as bs

# misc
import re as re # regex 
import time
import pandas as pd

# accesing env file 
import os 
from dotenv import load_dotenv # to access the secret keys we've hidden in a separate file 
load_dotenv() # grab values inside env file

True

#### Selenium and LinkedIn Setup

In [7]:
PATH = os.getenv("WEBDRIVER_PATH")
USERNAME = os.getenv("LI_USERNAME")
PASSWORD = os.getenv("LI_PASS")


In [8]:
# initialize web driver that would control the web browser
ser = Service(PATH)
op = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=ser, options=op)

# website we wanted to access 
driver.get("https://www.linkedin.com/uas/login")
time.sleep(3) # added a pause to avoid getting marked as bot 

# login in linkedin
email=driver.find_element(By.ID,"username")
email.send_keys(USERNAME)
password=driver.find_element(By.ID,"password")
password.send_keys(PASSWORD)
time.sleep(3)
password.send_keys(Keys.RETURN)

#### Web Scraping Process

In [11]:
user_links = []
user_names = []
user_location = []
user_bio = []

def user_find_scrape(search_info, max_page):
    time.sleep(5)
    
    # scroll mindlessly to avoid getting flagged as a bot
    start=time.time()
    lastHeight = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        newHeight = driver.execute_script("return document.body.scrollHeight")
        if newHeight == lastHeight:
            break
        lastHeight = newHeight
        end=time.time()
        if round(end-start)>20: # scrolling thru website - pretending we're not a bot for 20 seconds 
            break

    # type into search bar 
    search_bar = driver.find_element(By.CLASS_NAME, "search-global-typeahead__input")
    search_bar.send_keys(search_info)
    time.sleep(5)
    search_bar.send_keys(Keys.ENTER)
    time.sleep(5)

    # expand search result 
    try: 
        expand_result_click = driver.find_element(By.LINK_TEXT, "See all people results").click()
        time.sleep(5)
    except:
        print("Expand result failure")
        driver.quit()

    page = 0
    while page <= max_page: # how many search result page | each page has 10 users
        # find container for all names available in the current page  
        expanded_page_result = driver.page_source
        linkedin_soup = bs(expanded_page_result.encode("utf-8"), "html")
        linkedin_soup.prettify()
        huge_container = linkedin_soup.find("ul",{"class":"reusable-search__entity-result-list"})
        containers = huge_container.findAll("li",{"class":"reusable-search__result-container"})

        # iterate thru each of the name - grab each user's specified value and append each user's value in an array 
        iterations = 0
        nos = 5 # grab only 5 users on each page
        for user in containers: 
            try:
                user_name_box = user.find("span",{"class":"entity-result__title-text"})
                user_name_value = user_name_box.find("span",{"aria-hidden":"true"}).get_text()
                user_link_value = user_name_box.find("a",{"class":"app-aware-link"}, href=True)["href"]
                user_location_value = user.find("div",{"class":"entity-result__secondary-subtitle"}).get_text()
                user_bio_value = user.find("div",{"class":"entity-result__primary-subtitle"}).get_text()

                user_links.append(user_link_value)
                user_names.append(user_name_value)
                user_location.append(user_location_value)
                user_bio.append(user_bio_value)

                iterations += 1

                if(iterations==nos):
                    break

            except:
                print("There is an error fetching users. Initiating exit.")              
                driver.quit()

        # click next button - scroll down and wait for 30 seconds for the next button to load 
        try: 
            time.sleep(5)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(30)
            next_button_find = driver.find_element(By.CSS_SELECTOR, ".artdeco-pagination__button--next")
            next_button_find.click()
            page += 1
            time.sleep(5)
        except:
            print("Next button clicked error. Initiating exit.")
            driver.quit()
        
    

            

#### Execute scraping

In [12]:
user_find_scrape("data engineer",2) # 1st parameter - keyword | 2nd parameter - max_page, each result page only has 10 people 
driver.quit()

#### Storing scraped data into specified file format

In [13]:
data = {
    "Name": user_names,
    "Location": user_location,
    "Bio": user_bio,
    "Link": user_links
}

# save to csv 
df = pd.DataFrame(data)
df.head(5)
df.to_csv("user_find.csv", encoding='utf-8', index=False)

# save to excel 
# writer = pd.ExcelWriter("user_find.xlsx", engine='xlsxwriter')
# df.to_excel(writer, index =False)
# writer.save()

ModuleNotFoundError: No module named 'xlsxwriter'